In [0]:
#Import library
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
#Get the link
try:
  BASE_URL = 'https://sofifa.com/'
  response = requests.get(BASE_URL)
except Exception as err:
  print(f'ERROR: {err}')

In [0]:
soup = BeautifulSoup(response.text)

In [0]:
#Find player
players = soup.find_all('tr')

In [0]:
#An array for all features
ava, name, pos, age, ova, pot, team, val, wage = [], [], [], [], [], [], [], [], []
for i in range(1 ,len(players)):
    try:
        ava.append(players[i].img['data-src'])
        name.append(players[i].find('a', class_ = 'nowrap').string.strip())
        pos.append(players[i].find_all('a', href_ = '')[2].string.strip())
        age.append(players[i].find('td', class_ = 'col col-ae').string.strip())
        ova.append(players[i].find('td', class_ = 'col col-oa').string.strip())
        pot.append(players[i].find('td', class_ = 'col col-pt').string.strip())
        team.append(players[i].find_all('a', href_ = '')[-1].string.strip())
        val.append(players[i].find('td', class_ = 'col col-vl').string.strip())
        wage.append(players[i].find('td', class_ = 'col col-wg').string.strip())

    except:
        print('pass')

In [0]:
#Next page crawling
offset = '?offset='
for i in range(1, 1500):
  try:
    BASE_URL = BASE_URL+offset+str(60*i)
    response = requests.get(BASE_URL)
  except Exception as err:
    print(f'ERROR: {err}')

  soup = BeautifulSoup(response.text)
  players = soup.find_all('tr')
  for i in range(1 ,len(players)):
    try:
        ava.append(players[i].img['data-src'])
        name.append(players[i].find('a', class_ = 'nowrap').string.strip())
        pos.append(players[i].find_all('a', href_ = '')[2].string.strip())
        age.append(players[i].find('td', class_ = 'col col-ae').string.strip())
        ova.append(players[i].find('td', class_ = 'col col-oa').string.strip())
        pot.append(players[i].find('td', class_ = 'col col-pt').string.strip())
        team.append(players[i].find_all('a', href_ = '')[-1].string.strip())
        val.append(players[i].find('td', class_ = 'col col-vl').string.strip())
        wage.append(players[i].find('td', class_ = 'col col-wg').string.strip())

    except:
        print('pass')

In [0]:
#Save to json
d = list(zip(ava, name, pos, age, ova, pot, team, val, wage))

with open('players.json', 'w') as file:
    json.dump(d, file)

In [0]:
with open('players.json', 'r') as file:
    reloaded = json.load(file)

In [0]:
#Create dataframe
df = pd.read_json('/content/players.json')
df.columns = ['image', 'name', 'position', 'age', 'overall', 'potential', 'team', 'value', 'wage']

In [0]:
#Check first 5
df.head(5)

,image,name,position,age,overall,potential,team,value,wage
0,https://cdn.sofifa.org/players/4/20/212523.png,Anderson Talisca,CAM,25,83,88,Guangzhou Evergrande Taobao FC,€34.5M,€38K
1,https://cdn.sofifa.org/players/4/20/212198.png,Bruno Fernandes,CAM,24,85,88,Sporting CP,€49M,€24K
2,https://cdn.sofifa.org/players/4/20/206517.png,J. Grealish,CAM,23,78,84,Aston Villa,€14M,€50K
3,https://cdn.sofifa.org/players/4/20/254544.png,Reinier,CF,17,72,90,Real Madrid,€6M,€17K
4,https://cdn.sofifa.org/players/4/20/225953.png,S. Bergwijn,RW,21,82,87,PSV,€30M,€23K


In [0]:
#Check info
df.info()

In [0]:
#Check describe
df.describe()

In [0]:
#Delete symbol
def del_euro(serie):
  for i in range(len(serie)):
    serie[i] = serie[i].strip('€')

In [0]:
del_euro(df['value'])
del_euro(df['wage'])

In [0]:
#Change float to int
a = df['value']
for i in range(len(a)):
  if 'M' in a[i]:
    b = a[i].strip('M')
    a[i] = float(b)*1000000
  else:
    c = a[i].strip('K')
    a[i] = float(c) * 1000
    

In [0]:
# Change float to int
d = df['wage']
for i in range(len(d)):
  f = d[i].strip('K')
  d[i] = float(f) * 1000

In [0]:
#Change type to int64
df['value'] = df['value'].astype('int64')
df['wage'] = df['wage'].astype('int64')

In [0]:
#Check corr
df.corr()

In [0]:
#Show heatmap
sns.heatmap(df.corr())

In [0]:
#Delete not relate column
df.drop(['image', 'team'], axis = 1, inplace=True)   

In [0]:
#Plot the value and position
plt.figure(figsize = (15, 10))
plt.scatter(df['position'], df['value'], color = 'red')
plt.show()

In [0]:
plt.figure(figsize=(15,15))
plt.scatter(df['overall'], df['value'], color = 'red')